Setup environment

In [ ]:
python3 -m venv venv # setup environment
source venv/bin/activate # activate environment
pip install django # install django framework
pip install djangorestframework # install django rest framework

Create a new project name `newsapi`

In [ ]:
django-admin startproject newsapi

Create new app name `news`

In [ ]:
python manage.py startapp news

Add the app name into the `INSTALLED_APPS` array in `newsapi/settings.py` file

In [ ]:
# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    
    'news'
]

Add new class in `news/models.py` file


In [ ]:
from django.db import models
# Create your models here.
class Article(models.Model):
    author = models.CharField(max_length=50)
    title = models.CharField(max_length=120)
    description = models.CharField(max_length=200)
    body = models.TextField()
    location = models.CharField(max_length=120)
    publication_date = models.DateField()
    active = models.BooleanField(default=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    def __str__(self) -> str:
        return  f"{self.author} {self.title}"
    

Run the commands to make migrations in database

In [ ]:
python manage.py makemigrations
python manage.py migrate

Run command to create super user and setup `superuser account` in `admin page`
Command will create a `admin.py` file  in `news` folder

In [ ]:
python manage.py createsuperuser # command will create a `admin.py` file  in `news` folder

In `news/admin.py` add following code to register our models to `admin page`

In [ ]:
from newsapi.news import models

# Register your models here.
admin.site.register(models.Article)

Create new folder `api`, and new file `serializers.py`. Add these content to the file

In [ ]:
from rest_framework import serializers
from news.models import Article

class ArticleSerializer(serializers.Serializer):
    id = serializers.IntegerField(read_only=True)
    author = serializers.CharField()
    title = serializers.CharField()
    description = serializers.CharField()
    body = serializers.CharField()
    location = serializers.CharField()
    publication_date = serializers.DateField()
    active = serializers.BooleanField()
    created_at = serializers.DateTimeField(read_only=True)
    updated_at = serializers.DateTimeField(read_only=True)
    
    
    def create(self, validated_data):
        print(validated_data)
        Article.objects.create(**validated_data)
    
    def update(self, instance, validated_data):
        instance.author = validated_data.get('author', instance.author)
        instance.title = validated_data.get('author', instance.title)
        instance.description = validated_data.get('description', instance.description)
        instance.location = validated_data.get('location', instance.location)
        instance.publication_date = validated_data.get('publication_date', instance.publication_date)
        instance.active = validated_data.get('active', instance.active)
        instance.created_at = validated_data.get('created_at', instance.created_at)
        instance.updated_at = validated_data.get('updated_at', instance.updated_at)
        instance.save()
        
        return instance

Now, we use `shell` to interact with models

In [ ]:
python manage.py shell

Add `rest_framework` to `newsapi/newsapi/settings.py` `INSTALLED_APPS`

In [ ]:

# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    
    'news',
    'rest_framework'
]

Create new file `views.py` in `api` folder,  using `@api_view` decorator to indicate the method on api

In [ ]:
# views.py
from rest_framework import status
from rest_framework.decorators import api_view
from rest_framework.response import Response

from news.models import Article
from news.api.serializers import ArticleSerializer

@api_view(["GET", "POST"])
def article_list_create_api_view(request):
    if request.method == "GET":
        articles = Article.objects.filter(active=True)
        serializer = ArticleSerializer(articles, many=True)
        return Response(serializer.data)
    
    elif request.method == "POST":
        serializer = ArticleSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


Create new file `urls.py` in `api` folder

In [ ]:
# urls.py
from django.urls import path
from news.api.views import article_list_create_api_view

urlpatterns = [
    path("articles/", article_list_create_api_view, name="article-list")
    
]


Add new line `path("api/", include("news.api.urls"))` into `urlpatterns` of `newsapi/newsapi/urls.py`, then it will become:

In [ ]:
urlpatterns = [
    path('admin/', admin.site.urls),
    path("api/", include("news.api.urls"))
]


Let check the result in `http://127.0.0.1:8000/api/articles`


Now, add function to handle detail action `GET`, `PUT`, `DELETE`. Add following code to bottom of file `news/api/views.py`

In [ ]:

@api_view(["GET", "PUT", "DELETE"])
def article_detail_api_view(request, pk):
    try:
        article = Article.objects.get(pk=pk)
    except Article.DoesNotExist:
        return Response({"error": {
            "code": 404,
            "message": "Article not found!"
        }}, status=status.HTTP_404_NOT_FOUND)
    if request.method == "GET":
        serializer = ArticleSerializer(article)
        return Response(serializer.data)
    elif request.method == "PUT":
        serializer = ArticleSerializer(article, data=request.data)
        if (serializer.is_valid()):
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.data, status=status.HTTP_404_NOT_FOUND)
    elif request.method == "DELETE":
        article.delete()
        return Response(serializer.data, status=status.HTTP_204_NO_CONTENT)
        

Add new path ` path("articles/<int:pk>", article_detail_api_view, name="article-detail")` to handle new method to `api/urls.py` file

In [ ]:

urlpatterns = [
    path("articles/", article_list_create_api_view, name="article-list"),
    path("articles/<int:pk>", article_detail_api_view, name="article-detail"),
    
]

Now we  can use `ApiView` class to handle those apis separately in class, instead of methods. <br/>
In `api/views.py` we import 

In [ ]:
from rest_framework.views import APIView
from rest_framework.generics import get_object_or_404

Add below code right below the file `api/views.py` 

In [ ]:

class ArticleListApiView(APIView):
    def get(self, request):
        articles = Article.objects.filter(active=True)
        serializer = ArticleSerializer(articles, many=True)
        return Response(serializer.data)
    
    def post(self, request):
        serializer = ArticleSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
    
class ArticleDetailApiView(APIView):
    def get_object(self, pk) -> Article:
        article = get_object_or_404(Article, pk=pk)
        return article
    
    def get(self, request, pk):
        article = self.get_object(pk=pk)
        serializer = ArticleSerializer(article)
        
        return Response(serializer.data)
    
    def put(self, request, pk):
        article = self.get_object(pk)
        serializer = ArticleSerializer(article, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    
    def delete(self, request, pk):
        article = self.get_object(pk)
        article.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)
        

In `api/urls.py` we comment out the methods `article_list_create_api_view, article_detail_api_view` and using the classes api:

In [ ]:

urlpatterns = [
    # path("articles/", article_list_create_api_view, name="article-list"),
    # path("articles/<int:pk>", article_detail_api_view, name="article-detail"),
    path("articles/", ArticleListApiView.as_view(), name="article-list"),
    path("articles/<int:pk>", ArticleDetailApiView.as_view(), name="article-detail"),
    
]


Now we'll implement <h3>`Validation`</h3> to our apis. Import this to class `ArticleSerializer`. <br/>


In [ ]:
       
def validate(self, data):
    """check that description and title are different

    Args:
        data (_type_): _description_
    """
    
    if data["title"] == data["description"]:
        raise serializers.ValidationError("Title and Description must be different from one another.")
    return data

def validate_title(self, value):
    if len(value) < 60:
        raise serializers.ValidationError("Title must be at least 60 characters .") 
    return value + "_validated_"

The `def validate(self, data)` method, is the  validator of whole class `ArticleSerializer`.<br> On the other hand, `def validate_title(self, value)` is the validator of only `title` property. <br> Both these methods are built-in by `rest_framework`

Next, <h3>`ModelSerializer`</h3>
Instead of using above methods, we can use `ModelSerializer` class to speed up the Validation implementation.<br>
Replace `ArticleSerializer` with a new version of itself.

In [ ]:

class ArticleSerializer(serializers.ModelSerializer):
    class Meta:
        model = Article
        # fields = "__all__" # we want all the fields of our model to be serialized
        # fields = ("title", "description") # we want only some of fields of our model to be serialized
        exclude = ("id") # we want to serialize all of fields except "id"


We can add additional field to the Serializer class, so the `ArticleSerializer` can have a new field, separated from the origin `Article` class.<br>
After add the new field `time_since_publication` the class would be like this.

In [ ]:
from datetime import datetime
from django.utils.timesince import timesince

class ArticleSerializer(serializers.ModelSerializer):
    
    time_since_publication = serializers.SerializerMethodField(method_name="since_publication")
    
    class Meta:
        model = Article
        # fields = "__all__" # we want all the fields of our model to be serialized
        # fields = ("title", "description") # we want only some of fields of our model to be serialized
        exclude = ("id",) # we want to serialize all of fields except "id"

    def since_publication(self, object: Article) -> str: 
        publication_date = object.publication_date
        now = datetime.now()
        time_data = timesince(publication_date, now)
        return time_data
    

Next, we'll deal with the <h3>`Nested Relationships`</h3> objects. In `news/models.py` we create class `Journalist`.

In [ ]:
class Journalist(models.Model):
    first_name = models.CharField(max_length=60)
    last_name = models.CharField(max_length=60)
    biography = models.TextField(blank=True)
    
    
    def __str__(self) -> str:
        return f"{self.first_name} {self.last_name}"

Update the `Article` class on `author` property:

In [ ]:
...
class Article(models.Model):
    author = models.ForeignKey(Journalist, 
                               on_delete=models.CASCADE, 
                               related_name="articles")
    ...

In `news/admin.py` we also register the model to admin

In [ ]:
admin.site.register(models.Journalist)

Then we'll need to remove the current database file to create a new one. The Django will help us to create a new database file with empty content. So no need to worry. <br> After that, do the <b><i>`makemigrations`</i></b> rituals, <i>`creatsuperuser`</i>,...<br>
Now create a new class `JournalistSerializer` in `api/serializers.py`, above `ArticleSerializer` class

In [ ]:

class JournalistSerializer(serializers.ModelSerializer):
    
    class Meta:
        model = Journalist
        fields = "__all__" # we want all the fields of our model to be serialized
        

Add `author` as new custom field to `ArticleSerializer`. So now it would be like this:

In [ ]:
        
class ArticleSerializer(serializers.ModelSerializer):
    time_since_publication = serializers.SerializerMethodField(method_name="since_publication")
    author = JournalistSerializer()
    
    class Meta:
        model = Article
        # fields = "__all__" # we want all the fields of our model to be serialized
        # fields = ("title", "description") # we want only some of fields of our model to be serialized
        exclude = ("id",) # we want to serialize all of fields except "id"

    def since_publication(self, object: Article) -> str: 
        publication_date = object.publication_date
        now = datetime.now()
        time_data = timesince(publication_date, now)
        return time_data